In [1]:
import cv2
import numpy as np
import torch
from matplotlib import pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from IPython.display import display
import ipywidgets as widgets

In [2]:
model = YOLO("yolov11.pt")

In [3]:
tracker = DeepSort(max_age=30, n_init=3, nn_budget=100)

In [4]:
video_path = '15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

frames = []
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference with YOLO
    results = model(frame)
    detections = results[0].boxes.xyxy.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    filtered_dets = []
    for det, cls in zip(detections, classes):
        if int(cls) == 0:  # Class 0 = player
            x1, y1, x2, y2 = map(int, det[:4])
            w, h = x2 - x1, y2 - y1
            conf = float(det[4]) if len(det) > 4 else 0.8
            filtered_dets.append(([x1, y1, w, h], conf, 'player'))

    # Update Deep SORT
    tracks = tracker.update_tracks(filtered_dets, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 12), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_count += 1

cap.release()


0: 384x640 1 ball, 16 players, 2 referees, 1868.0ms
Speed: 4.3ms preprocess, 1868.0ms inference, 6.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 1739.9ms
Speed: 9.2ms preprocess, 1739.9ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 1743.2ms
Speed: 4.6ms preprocess, 1743.2ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 1751.9ms
Speed: 5.5ms preprocess, 1751.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 1756.5ms
Speed: 5.7ms preprocess, 1756.5ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 1752.8ms
Speed: 4.5ms preprocess, 1752.8ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 1752.1ms
Speed: 4.5ms preprocess, 1752.1ms inference, 3.3ms 

In [5]:
player_counts = []

def count_players_in_frame(frame):
    # Count number of 'ID:' labels to estimate players
    count = 0
    hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))  # green box color range
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return len(contours)

for f in frames:
    player_counts.append(count_players_in_frame(f))

def view_frame_with_graph(index):
    fig, axs = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [4, 1]})

    # Show the frame
    axs[0].imshow(frames[index])
    axs[0].set_title(f"Frame {index} with Player IDs")
    axs[0].set_xlabel("X-axis (pixels)")
    axs[0].set_ylabel("Y-axis (pixels)")
    axs[0].axis('on')

    # Bar chart of players across frames
    axs[1].bar(range(len(player_counts)), player_counts, color='lightblue')
    axs[1].set_title("Number of Players Detected per Frame")
    axs[1].set_xlabel("Frame Index")
    axs[1].set_ylabel("Player Count")
    axs[1].axvline(x=index, color='red', linestyle='--', label='Current Frame')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

slider = widgets.IntSlider(min=0, max=len(frames)-1, step=1, value=len(frames)//2, description='Frame')
widgets.interact(view_frame_with_graph, index=slider)

interactive(children=(IntSlider(value=187, description='Frame', max=374), Output()), _dom_classes=('widget-int…

<function __main__.view_frame_with_graph(index)>

In [6]:
if frames:
    height, width = frames[0].shape[:2]
    out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
    for f in frames:
        out.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
    out.release()
    print("✅ Output video saved as 'output_video.mp4'")
else:
    print("⚠️ No video to save.")

✅ Output video saved as 'output_video.mp4'
